In [3]:
from tqdm import tqdm

import torch
from torch import nn
from torch.utils.data import DataLoader

from data import DrawingDataset, collate_batch

# from mamba import Mamba, MambaConfig
from mamba_ssm import Mamba

In [4]:
train_dataset = DrawingDataset(data_path="./data", split="train", max_length=100)
val_dataset = DrawingDataset(data_path="./data", split="valid", max_length=100)
test_dataset = DrawingDataset(data_path="./data", split="test", max_length=100)

train = DataLoader(dataset=train_dataset, batch_size=64, shuffle=True, collate_fn=collate_batch)
val = DataLoader(dataset=val_dataset, batch_size=64, shuffle=True, collate_fn=collate_batch)
test = DataLoader(dataset=test_dataset, batch_size=64, shuffle=True, collate_fn=collate_batch)

100%|██████████| 345/345 [00:04<00:00, 73.38it/s]


In [5]:
# print(f"Length of train dataloader: {len(train)}")
# print(f"Length of validation dataloader: {len(val)}")
# print(f"Length of test dataloader: {len(test)}")

# for i,data in tqdm(enumerate(train)):
#     sketch, label = data
#     if i == 0:
#         print(sketch.shape, len(label))
#         print(sketch[0])
#     break

In [8]:
from mamba_ssm.models.config_mamba import MambaConfig

In [19]:
# print(sketch.shape, label)
# print(sketch[0][0].shape)
# model = nn.Sequential(
# )
# reshaped = torch.reshape(sketch, (64, 5, 100))
# print(reshaped.shape)
# print(reshaped[0])
config = MambaConfig(
    d_model = 5,
    # d_state = 16,
    # d_conv = 4,
    # expand=2
)
model = nn.Sequential(
    Mamba( d_model = 5, d_state = 16, d_conv = 4, expand=2),
    Mamba( d_model = 5, d_state = 16, d_conv = 4, expand=2),
    # Mamba( d_model = 5, d_state = 16, d_conv = 4, expand=2),
    # Mamba( d_model = 5, d_state = 16, d_conv = 4, expand=2),
    # Mamba( d_model = 5, d_state = 16, d_conv = 4, expand=2),
).to("cuda")

# print(sketch.dtype, sketch[0].size())
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-7)
# test = model(train[0][0].to("cuda"))
# print(test.shape)
# print(test[0][0])
# (B,L,D) - (64, 5, ???????????)
print_every = 1000
epochs = 5
for epoch in range(epochs):
    loss = None
    # for data in tqdm(val):
    for i, data in enumerate(train):
        inputs, targets = data
        inputs = inputs.to("cuda")
        targets = targets.to("cuda")
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()
        if i % print_every == 0:
            print(f"Epoch: {epoch+1}, Batch: {i}, Loss: {loss}")
    print(f"Epoch: {epoch+1}, Loss: {loss.item()}")

Epoch: 1, Batch: 0, Loss: 1.858309822489291e+35


KeyboardInterrupt: 